In [1]:
from ehrmonize import EHRmonize
import pandas as pd
import os
from dotenv import load_dotenv
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [11]:
import boto3
import json

bedrock = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": "Who are you?"}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")

response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))

None


In [3]:
prompt = "You are a well trained clinician doing data cleaning and harmonization. \
            You are given a raw drug name out of EHR, below, within squared brackets[]. \
            Please give me this drug's generic name in accordance with RxNorm standards: [0.9% Sodium Chloride] \
            Remove salt names unless there are multiple salts with different clinical effects. \
            Remove prescription strengths. \
            Include concentrations for intravenous fluids and dextrose solutions. \
            Please output nothing more than the generic name in lowercase."

In [26]:

load_dotenv()

aws_access_key_id = str(os.getenv('AWS_ACCESS_KEY_ID'))
aws_secret_access_key = str(os.getenv('AWS_SECRET_ACCESS_KEY'))

client = boto3.client(
            service_name = 'bedrock-runtime',
            region_name = 'us-east-1',
            aws_access_key_id = aws_access_key_id,
            aws_secret_access_key = aws_secret_access_key,
        )

body = json.dumps({
    "max_tokens": 32, 
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.1,
    "anthropic_version": "bedrock-2023-05-31"

})

response = bedrock.invoke_model(body = body,
                                    modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0",
                                            accept = 'application/json',
                                            contentType =  'application/json'
                                    )

response_body = json.loads(response.get('body').read())

response_body.get('content')[0]['text']

'sodium chloride 0.9%'

In [4]:
invoke_bedrock_claude('prompt')

NameError: name 'invoke_bedrock_claude' is not defined

In [7]:
ehrm = EHRmonize("gpt-4o", temperature=.1)

ehrm._invoke_openai(prompt)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [3]:
models = [
    # 'gpt-3.5-turbo',
    # 'gpt-4',
    'gpt-4o',
    # 'meta.llama2-13b-chat-v1',
    # 'meta.llama2-70b-chat-v1',
    # 'meta.llama3-70b-instruct-v1:0',
    # 'anthropic.claude-v2:1',
    # 'anthropic.claude-instant-v1',
    # 'mistral.mistral-7b-instruct-v0:2',
    # 'mistral.mixtral-8x7b-instruct-v0:1',
    # 'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [4]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=2,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='one_hot_drug_classification',
            classif='antibiotic'
      )

      pred = ehrm.predict(
            pd.DataFrame({
            'drug': ['ativan','vancocin'],
            'route': ['intravenous','oral']
        })
      )['pred']

      acc = accuracy_score(
            pd.Series(["0","1"]),
            pred
      )

      print(pred)
      print(acc)



Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
API failed, proceeding...
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
API failed, proceeding...
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
API failed, pro

In [4]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=2,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='custom',
            prompt='Please respond 1 if the medication below is an antibiotic, else 0:',
      )

      pred = ehrm.predict(
            pd.Series(['ativan','vancocin']),
      )['pred']

      acc = accuracy_score(
            pd.Series(["0","1"]),
            pred
      )

      print(pred)
      print(acc)



0    0
1    1
dtype: object
1.0


In [8]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=1,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='get_generic_name',
      )

      pred = ehrm.predict(
            pd.Series(['ativan','vancocin']),
      )

      acc = accuracy_score(
            pd.Series(['lorazepam','vancomycin']),
            pred
      )

      print(pred)
      print(acc)



0     lorazepam
1    vancomycin
dtype: object
1.0


In [9]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=2,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='get_generic_name',
      )

      pred = ehrm.predict(
            pd.Series(['ativan','vancocin']),
      )['pred']

      acc = accuracy_score(
            pd.Series(['lorazepam','vancomycin']),
            pred
      )

      print(pred)
      print(acc)



0     lorazepam
1    vancomycin
dtype: object
1.0


In [10]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=1,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='get_generic_route',
            classes=['intravenous', 'oral', 'other'],
      )

      pred = ehrm.predict(
            pd.Series(['intravenous','via mouth', 'rectal']),
      )

      acc = accuracy_score(
            pd.Series(['intravenous','oral','other']),
            pred
      )

      print(pred)
      print(acc)



0    intravenous
1           oral
2          other
dtype: object
1.0


In [4]:
for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config_prompting(
            n_attempts=2,
            agentic=False,
            n_shots=0,
      )

      ehrm.set_task(
            task='classify_drug',
            classes=['antibiotic', 'sedative', 'other'],
      )

      pred = ehrm.predict(
        pd.DataFrame({
            'drug': ['ativan','vancocin'],
            'route': ['intravenous','oral']
        })
      )

      acc = ehrm.evaluate(
            pd.Series(['sedative','antibiotic']),
            pred
      )

      print(pred)
      print(acc)



0      sedative
1    antibiotic
dtype: object
1.0


In [3]:
possible_shots = [
    "An input route 'intraven' would be classified as 'intravenous'.",
    "An input route 'intramuscul' would be classified as 'intramuscular'.",
    "An input route 'rEcTal' would be classified as 'rectal'.",
    "An input route 'oral' would be classified as 'oral'.",
    "An input route 'po' would be classified as 'oral'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config(
            n_attempts=3,
            agentic=False,
            n_shots=2,
      )
      
      results, decision, consistency = ehrm.get_generic_route(
            route='through the mouth',
            classes=['intravenous', 'rectal', 'oral','other'],
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: meta.llama3-70b-instruct-v1:0
Results: ['oral', 'oral', 'oral']
Decision: oral
Consistency: 1.00


In [4]:
possible_shots = [
    "An input drugname 'vancocin' would be classified as 'vancomycin'.",
    "An input drugname 'vancomycin' would be classified as 'vancomycin'.",
    "An input drugname 'Toprol XL' would be classified as 'metroprolol'.",
    "An input drugname 'Adderall' would be classified as 'amphetamine/dextroamphetamine'.",
    "An input drugname 'Tylenol' would be classified as 'acetaminophen'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config(
            n_attempts=3,
            agentic=False,
            n_shots=2,
      )

      results, decision, consistency = ehrm.get_generic_name(
            drugname='ativan',
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")

Model: meta.llama3-70b-instruct-v1:0
Results: ['lorazepam', 'lorazepam', 'lorazepam']
Decision: lorazepam
Consistency: 1.00


In [5]:
possible_shots = [
    "An input drugname 'vancocin' with a route 'intravenous' would be classified as 'antibiotic'.",
    "An input drugname 'vancomycin' with a route 'intravenous' would be classified as 'antibiotic'.",
    "An input drugname 'Toprol XL' with a route 'oral' would be classified as 'beta blocker'.",
    "An input drugname 'Tylenol' with a route 'oral' would be classified as 'analgesic'.",
    "An input drugname 'vasopressin' with a route 'intravenous' would be classified as 'vasopressor'.",
]

for m in models:
      ehrm = EHRmonize(m, temperature=.1)

      ehrm.config(
            n_attempts=3,
            agentic=False,
            n_shots=2,
      )

      results, decision, consistency = ehrm.classify_drug(
            drugname='dobutamine',
            route='intravenous',
            classes=['antibiotic', 'anticoagulant', 'other'],
            possible_shots=possible_shots,
          )

      print(f"Model: {m}")
      print(f"Results: {results}")
      print(f"Decision: {decision}")
      print(f"Consistency: {consistency:.2f}")
    

Model: meta.llama3-70b-instruct-v1:0
Results: ['other', 'other', 'other']
Decision: other
Consistency: 1.00
